Web Scraping for Job Listings
Objective:
In this exercise, you will use Web Scraping techniques to collect data from a job listing website, such
as Wuzzuf (or any similar job portal), and extract the following information:
1. Job Title
2. Company Name
3. Location
4. Job Type (Full-Time or Remote)
5. Required Experience
6. Job Description
7. Requirements

Goal:
Write a Python script that scrapes this data and stores it in a CSV or Excel file, where each record
contains this information for each job listing you extract.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
!pip install selenium
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
chrome_options = Options()
chrome_options.add_argument("--headless")  # تشغيل بدون واجهة
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 27.1 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,172 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugi

In [6]:
import csv
driver = webdriver.Chrome(options=chrome_options)
url = "https://wuzzuf.net/search/jobs/?q=data%20scientist%20&a=hpb"
driver.get(url)
time.sleep(3)
soup = BeautifulSoup(driver.page_source, "html.parser")
job_elements = soup.find_all("div", class_="css-1gatmva")
jobs_data = []
for job_element in job_elements:
    job_title = job_element.find("h2").text.strip() if job_element.find("h2").text.strip() else "N/A"
    company_name = job_element.find("a", class_="css-17s97q8").text.strip() if job_element.find("a", class_="css-17s97q8") else "N/A"
    location = job_element.find("span", class_="css-5wys0k").text.strip() if job_element.find("span", class_="css-5wys0k") else "N/A"
    link = job_element.find("a", class_="css-o171kl")["href"] if job_element.find("a", class_="css-o171kl")["href"] else "N/A"
    # New features for assignment
    work_time  = job_element.find("span", class_="css-1ve4b75").text.strip() if job_element.find("span", class_="css-1ve4b75") else "N/A"
    work_type = job_element.find("span", class_="css-o1vzmt").text.strip() if job_element.find("span", class_="css-o1vzmt") else "N/A" # online or on site or hybrid
    how_long_ago = job_element.find("div", class_="css-do6t5g").text.strip() if job_element.find("div", class_="css-do6t5g") else "N/A"
    jobs_data.append([job_title, company_name, location, link, work_time, work_type, how_long_ago])
    detailed_jobs_data=[]
for job in jobs_data:
    job_title, company_name, location, job_link, work_time, work_type, how_long_ago = job
    try:
        driver.get(job_link)
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        # New features for assignment
        description = soup.find("div", class_="css-1uobp1k").text.strip() if soup.find("div", class_="css-1uobp1k") else "N/A"
        requirements = soup.find("div", class_="css-1t5f0fr").text.strip() if soup.find("div", class_="css-1t5f0fr") else "N/A"
        elements = soup.find_all("span", class_="css-4xky9y")
        experience_needed = elements[0].text.strip() if len(elements) > 0 else "N/A"
        career_level = elements[1].text.strip() if len(elements) > 1 else "N/A"
        education_level = elements[2].text.strip() if len(elements) > 2 else "N/A"
        salary = elements[3].text.strip() if len(elements) > 3 else "N/A"
        detailed_jobs_data.append([job_title, company_name, location, work_time, work_type, how_long_ago, description, requirements, experience_needed, career_level, education_level, salary])
    except Exception as e:
        print ("failure to open link: ",e)
        continue
df = pd.DataFrame(detailed_jobs_data)
df.to_csv('detailed_jobs_data.csv', index=False)
